# NLP Class Activity 2: Documentation for Byte-Pair Encoding

### **Objective**
This notebook provides a step-by-step implementation of the **Byte-Pair Encoding (BPE)** algorithm from scratch.

### **How BPE Works: The Core Algorithm**
The BPE algorithm follows a simple, iterative process:
1.  **Initialize Vocabulary**: Start with a base vocabulary containing every individual character present in the text.
2.  **Find Most Frequent Pair**: Scan the entire text corpus and identify the pair of adjacent tokens that appears most frequently.
3.  **Merge the Pair**: Create a new token by merging this pair. Add this new token to your vocabulary.
4.  **Repeat**: Continue finding and merging the next most frequent pair for a predetermined number of iterations. With each merge, the vocabulary grows to include more complex subwords (e.g., from `t` and `h` to `th`, then from `th` and `e` to `the`).

In [1]:
from collections import Counter, defaultdict
import re

## **Code Implementation Breakdown**

### **Step 1: Load the Text Corpus**
The code begins by downloading the "Tiny Shakespeare" dataset, which is a single plain text file containing the complete works of Shakespeare. This text serves as our training data, from which the BPE algorithm will learn its vocabulary.

In [ ]:
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O tinyshakespeare.txt

with open("tinyshakespeare.txt", "r", encoding="utf-8") as f:
    corpus = f.read()

print("Corpus loaded!")
print("Corpus length (characters):", len(corpus))
print("Sample text:\n", corpus[:200])

### **Step 2: Create the Initial Vocabulary**
Before the main BPE process can start, we need to "pre-tokenize" the corpus. The `get_vocab` function does this:
* It takes each word from the text (e.g., "First").
* It splits the word into individual characters (`'F', 'i', 'r', 's', 't'`).
* It adds a special **end-of-word token** `</w>` to the end. This is crucial as it allows the model to understand where a word ends. The result for "First" would be `F i r s t </w>`.
* The initial vocabulary is a dictionary counting the frequency of each of these character-based words.

In [ ]:
def get_vocab(text):
    vocab = Counter()
    for word in text.split():
        vocab[" ".join(list(word)) + " </w>"] += 1
    return vocab

vocab = get_vocab(corpus[:10000])
print("\n Sample vocabulary entries (word -> frequency):")
for i, (w, f) in enumerate(vocab.items()):
    if i > 5: break
    print(w, ":", f)

### **Step 3: The BPE Loop (Finding and Merging Pairs)**
This is the core of the algorithm, implemented in the `byte_pair_encoding` function. It runs a loop for a set number of merges (in this case, 10 for the demo). In each iteration:

1.  **Get Pair Statistics (`get_stats`)**:
2.  **Find the Best Pair**:
3.  **Merge the Vocabulary (`merge_vocab`)**:

In [ ]:
def get_stats(vocab):
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs


def merge_vocab(pair, vocab):
    pattern = re.escape(' '.join(pair))
    pattern = re.compile(r'(?<!\S)' + pattern + r'(?!\S)')
    new_vocab = {}
    for word in vocab:
        new_word = pattern.sub(''.join(pair), word)
        new_vocab[new_word] = vocab[word]
    return new_vocab

### **Step 4: The Final Result**
After the loop completes, the script prints the list of `merges` that were learned. This list **is the learned BPE vocabulary**. Each tuple, like `('t', 'h')`, represents a merge rule. These learned subwords can now be used to tokenize new text. For example, a new, unseen word like "therefore" could be broken down into the known subwords `th`, `er`, `ef`, `or`, `e`.


In [ ]:
def byte_pair_encoding(vocab, num_merges=10):
    merges = []
    for i in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            break
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)
        merges.append(best)
        print(f"Step {i+1}: Merge {best}")
    return vocab, merges

final_vocab, merges = byte_pair_encoding(vocab, num_merges=15)

print(merges)